# Plotting jets as images

In [1]:
import time
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from awkward import JaggedArray, MaskedArray
import numpy as np
import glob as glob
import itertools
import json
import uproot_methods
import copy
from coord_transform import *
import tensorflow as tf
import pandas as pd
import math

import resource

np.set_printoptions(threshold=np.inf)

In [2]:
def pseudorapidity(theta):
    return -np.log(np.tan(theta*0.5))

In [3]:
def dataStream(output, fname):
    
    image = output['flippedImages'].value
    jetPt = output['jetPt'].value
    jetEta = output['jetEta'].value
    jetPhi = output['jetPhi'].value
    jetMass = output['jetMass'].value
    jetMassSD = output['jetMassSD'].value
    jetconstPt_log = output['jetconstPt_log'].value
    jetconstPt_Jetlog = output['jetconstPt_Jetlog'].value
    jetconstE_log = output['jetconstE_log'].value
    jetconstEta_abs = np.abs(output['jetconstEta'].value)
    delta_eta = output['delta_eta'].value
    delta_phi = output['delta_phi'].value
    deltaR_jet = output['deltaR_jet'].value
    deltaR_subjet0 = output['deltaR_subjet0'].value
    deltaR_subjet1 = output['deltaR_subjet1'].value
    charge = output['charge'].value
    isEle = output['isEle'].value
    isPho = output['isPho'].value
    isMuon = output['isMuon'].value
    isCh = output['isCh'].value
    isNh = output['isNh'].value
    dz = output['dz'].value
    dxy = output['dxy'].value
    labels = output['label'].value
    chMult = output['chargedHadronMult'].value
    neutMult = output['neutralHadronMult'].value
    phoMult = output['photonMult'].value
    eleMult = output['electronMult'].value
    muMult = output['muonMult'].value
    jetpull = output['pull_angle'].value
    deltaR_subjets = output['deltaR_subjets'].value
    z = output['z'].value

    jetTau1_b05 =  output['jetTau1_b0.5'].value
    jetTau2_b05 =  output['jetTau2_b0.5'].value
    jetTau3_b05 =  output['jetTau3_b0.5'].value
    jetTau1_sd_b05 =  output['jetTau1_sd_b0.5'].value
    jetTau2_sd_b05 =  output['jetTau2_sd_b0.5'].value
    jetTau3_sd_b05 =  output['jetTau3_sd_b0.5'].value
    jetTau1_b10 =  output['jetTau1_b1.0'].value
    jetTau2_b10 =  output['jetTau2_b1.0'].value
    jetTau3_b10 =  output['jetTau3_b1.0'].value
    jetTau1_sd_b10 =  output['jetTau1_sd_b1.0'].value
    jetTau2_sd_b10 =  output['jetTau2_sd_b1.0'].value
    jetTau3_sd_b10 =  output['jetTau3_sd_b1.0'].value
    jetTau1_b15 =  output['jetTau1_b1.5'].value
    jetTau2_b15 =  output['jetTau2_b1.5'].value
    jetTau3_b15 =  output['jetTau3_b1.5'].value
    jetTau1_sd_b15 =  output['jetTau1_sd_b1.5'].value
    jetTau2_sd_b15 =  output['jetTau2_sd_b1.5'].value
    jetTau3_sd_b15 =  output['jetTau3_sd_b1.5'].value
    jetTau1_b20 =  output['jetTau1_b2.0'].value
    jetTau2_b20 =  output['jetTau2_b2.0'].value
    jetTau3_b20 =  output['jetTau3_b2.0'].value
    jetTau1_sd_b20 =  output['jetTau1_sd_b2.0'].value
    jetTau2_sd_b20 =  output['jetTau2_sd_b2.0'].value
    jetTau3_sd_b20 =  output['jetTau3_sd_b2.0'].value    

    jetFile = '/mnt/data/ml/ShowJetsData_' + fname
    constituentFile = '/mnt/data/ml/Constituent4vec_' + fname
      

    np.savez(jetFile, jetImages = image, jetPt = jetPt, jetEta = jetEta, jetPhi = jetPhi, 
             jetMass = jetMass, jetMassSD = jetMassSD, deltaR_subjets = deltaR_subjets, z = z,
             tau1_b05=jetTau1_b05, tau2_b05=jetTau2_b05, tau3_b05=jetTau3_b05, 
             tau1_sd_b05=jetTau1_sd_b05, tau2_sd_b05=jetTau2_sd_b05,tau3_sd_b05=jetTau3_sd_b05,
             tau1_b10=jetTau1_b10, tau2_b10=jetTau2_b10, tau3_b10=jetTau3_b10, 
             tau1_sd_b10=jetTau1_sd_b10, tau2_sd_b10=jetTau2_sd_b10,tau3_sd_b10=jetTau3_sd_b10,
             tau1_b15=jetTau1_b15, tau2_b15=jetTau2_b15, tau3_b15=jetTau3_b15, 
             tau1_sd_b15=jetTau1_sd_b15, tau2_sd_b15=jetTau2_sd_b15,tau3_sd_b15=jetTau3_sd_b15,
             tau1_b20=jetTau1_b20, tau2_b20=jetTau2_b20, tau3_b20=jetTau3_b20, 
             tau1_sd_b20=jetTau1_sd_b20, tau2_sd_b20=jetTau2_sd_b20,tau3_sd_b20=jetTau3_sd_b20,             
             chMult=chMult, neutMult = neutMult, phoMult=phoMult, eleMult=eleMult,
             muMult=muMult, jetpull=jetpull, labels = labels)
    
#     constituent_labels = np.repeat(labels, 20, axis=0).reshape(len(labels)*20, 3)
#     jetpull = np.repeat(jetpull, 20, axis=0).reshape(len(jetpull)*20)

#     jetTau1_b05 =  np.repeat(jetTau1_b05, 20, axis=0).reshape(len(jetTau1_b05)*20)
#     jetTau2_b05 =  np.repeat(jetTau2_b05, 20, axis=0).reshape(len(jetTau2_b05)*20)
#     jetTau3_b05 =  np.repeat(jetTau3_b05, 20, axis=0).reshape(len(jetTau3_b05)*20)
#     jetTau1_sd_b05 =  np.repeat(jetTau1_sd_b05, 20, axis=0).reshape(len(jetTau1_sd_b05)*20)
#     jetTau2_sd_b05 =  np.repeat(jetTau2_sd_b05, 20, axis=0).reshape(len(jetTau2_sd_b05)*20)
#     jetTau3_sd_b05 =  np.repeat(jetTau3_sd_b05, 20, axis=0).reshape(len(jetTau3_sd_b05)*20)
#     jetTau1_b10 =  np.repeat(jetTau1_b10, 20, axis=0).reshape(len(jetTau1_b10)*20)
#     jetTau2_b10 =  np.repeat(jetTau2_b10, 20, axis=0).reshape(len(jetTau2_b10)*20)
#     jetTau3_b10 =  np.repeat(jetTau3_b10, 20, axis=0).reshape(len(jetTau3_b10)*20)
#     jetTau1_sd_b10 =  np.repeat(jetTau1_sd_b10, 20, axis=0).reshape(len(jetTau1_sd_b10)*20)
#     jetTau2_sd_b10 =  np.repeat(jetTau2_sd_b10, 20, axis=0).reshape(len(jetTau2_sd_b10)*20)
#     jetTau3_sd_b10 =  np.repeat(jetTau3_sd_b10, 20, axis=0).reshape(len(jetTau3_sd_b10)*20)
#     jetTau1_b15 =  np.repeat(jetTau1_b15, 20, axis=0).reshape(len(jetTau1_b15)*20)
#     jetTau2_b15 =  np.repeat(jetTau2_b15, 20, axis=0).reshape(len(jetTau2_b15)*20)
#     jetTau3_b15 =  np.repeat(jetTau3_b15, 20, axis=0).reshape(len(jetTau3_b15)*20)
#     jetTau1_sd_b15 =  np.repeat(jetTau1_sd_b15, 20, axis=0).reshape(len(jetTau1_sd_b15)*20)
#     jetTau2_sd_b15 =  np.repeat(jetTau2_sd_b15, 20, axis=0).reshape(len(jetTau2_sd_b15)*20)
#     jetTau3_sd_b15 =  np.repeat(jetTau3_sd_b15, 20, axis=0).reshape(len(jetTau3_sd_b15)*20)
#     jetTau1_b20 =  np.repeat(jetTau1_b20, 20, axis=0).reshape(len(jetTau1_b20)*20)
#     jetTau2_b20 =  np.repeat(jetTau2_b20, 20, axis=0).reshape(len(jetTau2_b20)*20)
#     jetTau3_b20 =  np.repeat(jetTau3_b20, 20, axis=0).reshape(len(jetTau3_b20)*20)
#     jetTau1_sd_b20 =  np.repeat(jetTau1_sd_b20, 20, axis=0).reshape(len(jetTau1_sd_b20)*20)
#     jetTau2_sd_b20 =  np.repeat(jetTau2_sd_b20, 20, axis=0).reshape(len(jetTau2_sd_b20)*20)
#     jetTau3_sd_b20 =  np.repeat(jetTau3_sd_b20, 20, axis=0).reshape(len(jetTau3_sd_b20)*20)             

    

    np.savez(constituentFile, jetconstPt_log = jetconstPt_log, jetconstEta_abs = jetconstEta_abs,
             jetconstE_log = jetconstE_log, jetconstPt_Jetlog = jetconstPt_Jetlog,
             charge=charge, isEle = isEle,isPho = isPho, isMuon=isMuon,
             isCh=isCh, isNh=isNh, delta_eta=delta_eta, delta_phi = delta_phi, deltaR_jet = deltaR_jet,
             deltaR_subjet0 = deltaR_subjet0, deltaR_subjet1=deltaR_subjet1, dxy=dxy,
             dz=dz, labels = labels)

In [4]:
def transform_to_softdrop_axes(jetparticles,Jets):
    
    subjet0leading = (Jets.subjet0_pt > Jets.subjet1_pt)
    centroid1_pt = np.where( subjet0leading, Jets.subjet0_pt, Jets.subjet1_pt  )
    centroid1_eta = np.where( subjet0leading, Jets.subjet0_eta, Jets.subjet1_eta  )
    centroid1_phi = np.where( subjet0leading, Jets.subjet0_phi, Jets.subjet1_phi  )
    centroid1_m = np.where( subjet0leading, Jets.subjet0_m, Jets.subjet1_m  )
    centroid2_pt = np.where( subjet0leading, Jets.subjet1_pt, Jets.subjet0_pt,   )
    centroid2_eta = np.where( subjet0leading, Jets.subjet1_eta, Jets.subjet0_eta  )
    centroid2_phi = np.where( subjet0leading, Jets.subjet1_phi, Jets.subjet0_phi  )
    centroid2_m = np.where( subjet0leading, Jets.subjet1_m, Jets.subjet0_m  )
    pcentroid1 = uproot_methods.TLorentzVectorArray.from_ptetaphim( centroid1_pt, centroid1_eta, centroid1_phi, centroid1_m )
    pcentroid2 = uproot_methods.TLorentzVectorArray.from_ptetaphim( centroid2_pt, centroid2_eta, centroid2_phi, centroid2_m )
    
    #distance between the two centroids
    r0 = pcentroid1.delta_r(pcentroid2)
    
    centroid1 = uproot_methods.TVector3Array.from_cartesian(pcentroid1.x, pcentroid1.y, pcentroid1.z)
    centroid2 = uproot_methods.TVector3Array.from_cartesian(pcentroid2.x, pcentroid2.y, pcentroid2.z)
    particle_p3s = uproot_methods.TVector3Array.from_cartesian(jetparticles.p4.x, jetparticles.p4.y, jetparticles.p4.z)
    
    # Rotate coordinate system so "centroid1" is along the x axis
    rot1_axis, rot1_angle = coord_rotation( centroid1, uproot_methods.TVector3(1,0,0) )    
    centroid1 = centroid1.rotate_axis(rot1_axis, rot1_angle)
    centroid2 = centroid2.rotate_axis(rot1_axis, rot1_angle)
    particle_p3s = particle_p3s.rotate_axis(rot1_axis, rot1_angle)

    # Rotate about the z axis so that centroid2 is pointing down
    rot2_angle = -np.arctan2( centroid2.z, centroid2.y )
    centroid2 = centroid2.rotatex(rot2_angle)
    particle_p3s = particle_p3s.rotatex(rot2_angle)
    
    # Scale the image so that centroid2 is at (0,1,0)
    deta = np.divide(pseudorapidity(particle_p3s.theta),r0)
    dphi = np.divide(particle_p3s.phi,r0)
        
    return deta,dphi,centroid1,centroid2

In [5]:
# Look at ProcessorABC to see the expected methods and what they are supposed to do
class JetImageProcessor(processor.ProcessorABC):
    def __init__(self, storeEventByEvent=False):
        self.eta_bins = 16
        self.phi_bins = 16
        self.tau_betas = [0.5,1.0,1.5,2.0]
        dataset_axis = hist.Cat("dataset", "Primary dataset")        
        pdgid_axis = hist.Bin("pdgid", r"PDG ID", 10000, -5000, 5000)
        eta_axis = hist.Bin("eta", r"$\eta$", 32, -5, 5)
        phi_axis = hist.Bin("phi", r"$\phi$", 32, -np.pi, np.pi)
        image_dphi = hist.Bin("image_dphi", r"$Q_1$", self.phi_bins, -2, 2) #yaxis
        image_deta = hist.Bin("image_deta", r"$Q_2$", self.eta_bins, -2, 2) #xaxis
        pt_axis = hist.Bin("pt", r"$p_{T}$ [GeV]", 100, 0, 5000)
        m_axis = hist.Bin("m", r"$p_{T}$ [GeV]", 200, 0, 500)
        dr_axis = hist.Bin("dr", r"$\Delta r$", 500, 0, 5.0)
        drb_axis = hist.Bin("drb", r"$\Delta r_b$", 100, 0, 1.0)
        dxy_axis = hist.Bin("dxy", r"$d_{xy}$", 100, 0, 1.0)
        dz_axis = hist.Bin("dz", r"$d_z$", 500, -5.0, 5.0)
        hadron_axis = hist.Cat("hadron", "Hadron Category") # hist.Cat -- list of categories
        tau_axis = hist.Bin("tau", r"$\tau$", 100, 0, 1.0)
        mult_axis = hist.Bin("mult", "Multiplicity", 50, 0, 50)
        pullAngle_axis = hist.Bin("pullAngle", "Jet Pull Angle", 50, -math.pi, math.pi)
        pull_axis = hist.Bin("pull", "Jet Pull", 100, 0, 0.1)

        if storeEventByEvent:
            event_axis = hist.Bin("event", "Event number", 10000, 0, 10000 )
        else: 
            event_axis = hist.Bin("event", "Event number", 1, 0, 10000 )
        
        self._accumulator = processor.dict_accumulator({
            #hists for plotting
            'pt':hist.Hist("Counts", dataset_axis, pt_axis),
            'eta':hist.Hist("Counts", dataset_axis, eta_axis),
            'h_hadmult':hist.Hist("Counts", dataset_axis, mult_axis, hadron_axis),# event_axis),
            'h_muMult':hist.Hist("Counts", dataset_axis, mult_axis),
            'h_eleMult':hist.Hist("Counts", dataset_axis, mult_axis),
            'h_phoMult':hist.Hist("Counts", dataset_axis, mult_axis),
            'h_jetPull':hist.Hist("Counts", dataset_axis, pull_axis),
            'h_jetPullAngle':hist.Hist("Counts", dataset_axis, pullAngle_axis),
            'image':hist.Hist(r"$p_{T}$", dataset_axis, event_axis, image_dphi, image_deta),
            'dr':hist.Hist("Counts", dataset_axis, dr_axis),
            'h_drjb':hist.Hist("Counts", dataset_axis, dr_axis),
            'h_drjbb':hist.Hist("Counts", dataset_axis, dr_axis),
            'h_drbb':hist.Hist("Counts", dataset_axis, dr_axis),
            'h_dxy':hist.Hist("Counts", dataset_axis, dxy_axis),
            'h_dz':hist.Hist("Counts", dataset_axis, dz_axis),
            'pdgid':hist.Hist("Counts", dataset_axis, pdgid_axis),
            'tau21_b0.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau31_b0.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau32_b0.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau21_b1.0':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau31_b1.0':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau32_b1.0':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau21_b1.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau31_b1.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau32_b1.5':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau21_b2.0':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau31_b2.0':hist.Hist("Counts", dataset_axis, tau_axis),
            'tau32_b2.0':hist.Hist("Counts", dataset_axis, tau_axis),
            
            #Column Accumulators for ML
            'flippedImages': processor.column_accumulator(np.zeros(shape=(0,16,16))),
            'jetEta': processor.column_accumulator(np.zeros(shape=(0))),
            'jetPhi': processor.column_accumulator(np.zeros(shape=(0))),
            'jetPt': processor.column_accumulator(np.zeros(shape=(0))),
            'jetMass': processor.column_accumulator(np.zeros(shape=(0))),
            'jetMassSD': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau1_b0.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_b0.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_b0.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau1_sd_b0.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_sd_b0.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_sd_b0.5': processor.column_accumulator(np.zeros(shape=(0))),           
            'jetTau1_b1.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_b1.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_b1.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau1_sd_b1.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_sd_b1.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_sd_b1.0': processor.column_accumulator(np.zeros(shape=(0))), 
            'jetTau1_b1.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_b1.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_b1.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau1_sd_b1.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_sd_b1.5': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_sd_b1.5': processor.column_accumulator(np.zeros(shape=(0))),            
            'jetTau1_b2.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_b2.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_b2.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau1_sd_b2.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau2_sd_b2.0': processor.column_accumulator(np.zeros(shape=(0))),
            'jetTau3_sd_b2.0': processor.column_accumulator(np.zeros(shape=(0))),                     
            'chargedHadronMult': processor.column_accumulator(np.zeros(shape=(0))),
            'neutralHadronMult': processor.column_accumulator(np.zeros(shape=(0))),
            'photonMult': processor.column_accumulator(np.zeros(shape=(0))),
            'electronMult': processor.column_accumulator(np.zeros(shape=(0))),
            'muonMult': processor.column_accumulator(np.zeros(shape=(0))), 
            'label': processor.column_accumulator(np.zeros(shape=(0,3))),
            'particleType':processor.column_accumulator(np.zeros(shape=(0))),
            'jetconstEvnum': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstEta': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstPhi': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstPt': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstE': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstPt_log': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstPt_Jetlog': processor.column_accumulator(np.zeros(shape=(0,20))),
            'jetconstE_log': processor.column_accumulator(np.zeros(shape=(0,20))),
            'delta_eta': processor.column_accumulator(np.zeros(shape=(0,20))),
            'delta_phi': processor.column_accumulator(np.zeros(shape=(0,20))),
            'deltaR_jet': processor.column_accumulator(np.zeros(shape=(0,20))),
            'deltaR_subjet0': processor.column_accumulator(np.zeros(shape=(0,20))),
            'deltaR_subjet1': processor.column_accumulator(np.zeros(shape=(0,20))),
            'charge': processor.column_accumulator(np.zeros(shape=(0,20))),
            'isEle': processor.column_accumulator(np.zeros(shape=(0,20))),
            'isPho': processor.column_accumulator(np.zeros(shape=(0,20))),
            'isMuon': processor.column_accumulator(np.zeros(shape=(0,20))),
            'isCh': processor.column_accumulator(np.zeros(shape=(0,20))),
            'isNh': processor.column_accumulator(np.zeros(shape=(0,20))),
            'dz': processor.column_accumulator(np.zeros(shape=(0,20))),
            'dxy': processor.column_accumulator(np.zeros(shape=(0,20))),
            'pull_angle':processor.column_accumulator(np.zeros(shape=(0))),
            'deltaR_subjets':processor.column_accumulator(np.zeros(shape=(0))),
            'z': processor.column_accumulator(np.zeros(shape=(0))),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, df):
        
        output = self.accumulator.identity()

        dataset = df['dataset']
        evnum = df['eventNum']
        
        GenParts = JaggedCandidateArray.candidatesfromcounts(
            df['nGen'],
            pt=df['gen_pt'],
            eta=df['gen_eta'],
            phi=df['gen_phi'],
            mass=df['gen_m'],
            pdgId=df['gen_id'],
            flags=df['gen_flags'] #0: visible, 2: final parton level, 4: final state parton, 8: hadron
        )
        
        Jets = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt=df['jet_pt'],
            eta=df['jet_eta'],
            phi=df['jet_phi'],
            mass=df['jet_m'],
            msd=df['jet_msd'],
            nsubjet=df['jet_nsubjet'],
            subjet0_pt=df['jet_subjet0_pt'],
            subjet0_eta=df['jet_subjet0_eta'],
            subjet0_phi=df['jet_subjet0_phi'],
            subjet0_m=df['jet_subjet0_m'],
            subjet1_pt=df['jet_subjet1_pt'],
            subjet1_eta=df['jet_subjet1_eta'],
            subjet1_phi=df['jet_subjet1_phi'],
            subjet1_m=df['jet_subjet1_m'],
            tau1=df['jet_tau1'],
            tau2=df['jet_tau2'],
            tau3=df['jet_tau3'],
            tau4=df['jet_tau4'],
            tau1_sd=df['jet_tau1_sd'],
            tau2_sd=df['jet_tau2_sd'],
            tau3_sd=df['jet_tau3_sd'],
            tau4_sd=df['jet_tau4_sd'],
            vyy=df['gen_vyy'],
            vzz=df['gen_vzz'],
            ic=df['jet_ic'],
            nc=df['jet_nc']
            )        
        constituents = JaggedCandidateArray.candidatesfromcounts(
            df['nConstituent'],
            pt=df['constituent_pt'],
            eta=df['constituent_eta'],
            phi=df['constituent_phi'],
            mass=df['constituent_m'],
            pdgid=df['constituent_id'],
            status=df['constituent_status'],
            jetndx=df['constituent_jetndx'], # constituents belonging to the leading jet
            subjetndx=df['constituent_subjetndx'], # constituents belonging to the subjets
            vxx=df['constituent_vxx'],
            vyy=df['constituent_vyy'],
            vzz=df['constituent_vzz'],
            )
        
        deta,dphi,centroid1,centroid2= transform_to_softdrop_axes(constituents, Jets[:,0])        
        constituents.add_attributes( deta=deta )
        constituents.add_attributes( dphi=dphi )
        
        #Applying cuts to Jets and Jet Constituents
        fscut = (constituents.status == 91)
        constituents = constituents[fscut] # only final state particles in the jet
        constituents = constituents[constituents.pt > 1]
        jetconstituents = constituents[constituents.jetndx == 0]
        
        # Make a Jagged Array of event numbers with the correct offsets
        jaggedshape = JaggedArray.fromoffsets( jetconstituents.offsets, np.ones(deta.flatten().shape[0]))
        eventReshaped = jaggedshape * evnum 
        
        if ("ZZbbbb" in dataset):
            #find the b quarks for Z->bb
            bQuarks = GenParts[np.abs(GenParts.pdgId)==5]

            # Make sure there are at least two b quarks in the event
            twoBQuarksInEvent = bQuarks.counts >= 2
            bQuarks = bQuarks[twoBQuarksInEvent]
            Jets = Jets[twoBQuarksInEvent]
            evnum = evnum[twoBQuarksInEvent] 
            eventReshaped = eventReshaped[twoBQuarksInEvent]
            jetconstituents = jetconstituents[twoBQuarksInEvent]
            
            matches = Jets.cross( bQuarks )
            dRMatch_jb = matches.i0.p4.delta_r( matches.i1.p4 )
            dRCut = (dRMatch_jb < 0.8)
            dRMatch_jb = dRMatch_jb[dRCut]
            
            twoBQuarksInJet = dRCut.counts >= 2
            Jets = Jets[twoBQuarksInJet]
            evnum = evnum[twoBQuarksInJet]            
            eventReshaped = eventReshaped[twoBQuarksInJet]            
            jetconstituents = jetconstituents[twoBQuarksInJet]
        
        #Require at least one jet
        Jets = Jets[Jets.pt > 200]
        oneJet = (Jets.counts >=1)
        Jets = Jets[oneJet]
        jetconstituents = jetconstituents[oneJet]
        eventReshaped = eventReshaped[oneJet]

        #Require at least two subjets
        twoSubjets = (Jets[:,0].nsubjet >= 2)
        Jet0 = Jets[:,0][twoSubjets]
        jetconstituents = jetconstituents[twoSubjets]
        eventReshaped = eventReshaped[twoSubjets]
        dr = Jet0.p4.delta_r(jetconstituents.p4)
        
        validImages = np.zeros(int(np.max(evnum)+1), dtype=bool)
        evnum = evnum[oneJet]
        evnum = evnum[twoSubjets]
        validImages[evnum] = True
        
        deta = jetconstituents.deta
        dphi = jetconstituents.dphi
        
        # Performing parity flip so that sum(right) is higher than sum(left) for each image
        eventbins = np.arange(len(validImages)+1)
        xbins= np.linspace(-2,2,num=self.eta_bins+1)
        ybins= np.linspace(-2,2,num=self.phi_bins+1)
        
        # For the images, normalize each constituent to the jet pt
        # to remove any pt dependence
        const_pts = jetconstituents.pt
        normed_pts = const_pts / Jet0.p4.pt
        #print('normed_pts = ', normed_pts)
    
        H_images = np.histogramdd((eventReshaped.flatten(),dphi.flatten(),deta.flatten()),
                                  bins=(eventbins ,ybins, xbins), weights = normed_pts.flatten())[0][validImages]
        
        #H_images = H_images[validImages]
        mask = (np.sum(H_images[:,:,:int(self.eta_bins/2)], axis = (1,2)) < np.sum(H_images[:,:,int(self.eta_bins/2):], axis = (1,2)))
        
        sign = np.where(mask, 1, -1)
        deta = (sign*deta)        
        H_images_new = np.histogramdd((eventReshaped.flatten(),dphi.flatten(),deta.flatten()),
                                      bins=(eventbins ,ybins, xbins), weights = normed_pts.flatten())[0]

        labels = np.zeros([len(deta),3])
        if ("ZZbbbb" in dataset):
            labels[:,2]=1
        elif ("ZZ" in dataset):
            labels[:,1]=1
        else:
            labels[:,0]=1
            
        # Get the leading 20 constituents in each event for
        # output to a npz file for deepak8 like architechtures. 
        truncated_constituents = jetconstituents[jetconstituents.pt.argsort()][:,:20]
            
        # Adding the ch, nh, e, mu, gamma multiplicity information   
        isEle = (abs(jetconstituents.pdgid) == 11)
        isMuon = (abs(jetconstituents.pdgid) == 13)
        isPho = (abs(jetconstituents.pdgid) == 22)
        isCh = ((abs(jetconstituents.pdgid) == 2212) | (abs(jetconstituents.pdgid) == 211) |
                (abs(jetconstituents.pdgid) == 321))
        isNh = (abs(jetconstituents.pdgid) == 2112) | (abs(jetconstituents.pdgid) == 130)
        
        t_isEle = (abs(truncated_constituents.pdgid) == 11)
        t_isMuon = (abs(truncated_constituents.pdgid) == 13)
        t_isPho = (abs(truncated_constituents.pdgid) == 22)
        t_isCh = ((abs(truncated_constituents.pdgid) == 2212) | (abs(truncated_constituents.pdgid) == 211) |
                  (abs(truncated_constituents.pdgid) == 321))
        t_isNh = (abs(truncated_constituents.pdgid) == 2112) | (abs(truncated_constituents.pdgid) == 130)
        
        minus = ((truncated_constituents.pdgid == 11) | (truncated_constituents.pdgid == 13) |
                (truncated_constituents.pdgid == 15) | (truncated_constituents.pdgid == -2212) |
                (truncated_constituents.pdgid == -211) | (truncated_constituents.pdgid == -321))
        
        plus = ((truncated_constituents.pdgid == -11) | (truncated_constituents.pdgid == -13) |
                (truncated_constituents.pdgid == -15) | (truncated_constituents.pdgid == 2212) |
                (truncated_constituents.pdgid == 211) | (truncated_constituents.pdgid == 321))
        
        zero = ((truncated_constituents.pdgid == 22) | (truncated_constituents.pdgid == -22) |
                (truncated_constituents.pdgid == 2112) | (truncated_constituents.pdgid == -2112) |
                (truncated_constituents.pdgid == 130) | (truncated_constituents.pdgid == -130) | 
                (truncated_constituents.pdgid == 12) | (truncated_constituents.pdgid == -12) |
                (truncated_constituents.pdgid == 14) | (truncated_constituents.pdgid == -14) |
                (truncated_constituents.pdgid == 16) | (truncated_constituents.pdgid == -16))
        
        charge = JaggedArray.ones_like(zero)*(np.nan)
        charge[plus] = +1
        charge[minus] = -1
        charge[zero] = 0
        
        ele = np.char.multiply('e',pd.DataFrame((abs(truncated_constituents.pdgid) == 11)).fillna(False).values.astype(int))
        muon = np.char.multiply('mu',pd.DataFrame((abs(truncated_constituents.pdgid) == 13)).fillna(False).values.astype(int))
        gamma = np.char.multiply('gamma',pd.DataFrame((abs(truncated_constituents.pdgid) == 22)).fillna(False).values.astype(int))
        ch = np.char.multiply('ch',pd.DataFrame((abs(truncated_constituents.pdgid) == 2212) | (abs(truncated_constituents.pdgid) == 211) | (abs(truncated_constituents.pdgid) == 321)).fillna(False).values.astype(int))
        nh = np.char.multiply('nh',pd.DataFrame((abs(truncated_constituents.pdgid) == 2112) | (abs(truncated_constituents.pdgid) == 130)).fillna(False).values.astype(int))
        particleType = np.char.add((np.char.add((np.char.add(ele,gamma)),(np.char.add(ch,nh)))),muon)
        
        # Adding Jet Pull information
        SubJet0 = uproot_methods.TLorentzVectorArray.from_ptetaphim( Jet0.subjet0_pt, Jet0.subjet0_eta, Jet0.subjet0_phi, Jet0.subjet0_m )
        SubJet1 = uproot_methods.TLorentzVectorArray.from_ptetaphim( Jet0.subjet1_pt, Jet0.subjet1_eta, Jet0.subjet1_phi, Jet0.subjet1_m )

        SubJet0_constituents = jetconstituents[(jetconstituents.jetndx == 0) & (jetconstituents.subjetndx == 0)]
        SubJet1_constituents = jetconstituents[(jetconstituents.jetndx == 0) & (jetconstituents.subjetndx == 1)]

        # Calculating jet pull
        constituent_pt = SubJet0_constituents.pt
        constituent_y = SubJet0_constituents.p4.rapidity
        constituent_phi = SubJet0_constituents.p4.phi
        
        pull_dphi = SubJet1.delta_phi(SubJet0)
        pull_dy = SubJet1.rapidity - SubJet0.rapidity
        pull_jet_angle = np.arctan2(pull_dphi, pull_dy)

        pull_i_r = SubJet0_constituents.p4.delta_r(SubJet0)
        pull_i_dphi = SubJet0_constituents.p4.delta_phi(SubJet0)
        pull_i_dy = SubJet0_constituents.p4.rapidity - SubJet0.rapidity
        
        # Calculate phi and y components of pull
        valtemp = np.divide( np.multiply(constituent_pt, pull_i_r), Jet0.p4.pt )
        pull_tosum_phi = np.multiply(valtemp, pull_i_dphi)
        pull_tosum_y = np.multiply(valtemp, pull_i_dy)
        
        pull_sum_phi = pull_tosum_phi.sum()
        pull_sum_y = pull_tosum_y.sum()
        pull_mag = np.sqrt(np.power(pull_sum_phi,2) + np.power(pull_sum_y,2))
        
        # Calculate pull angle in polar coordinates
        pull_angle = (np.arctan2(pull_sum_phi, pull_sum_y) - pull_jet_angle + math.pi) % (2*math.pi) - math.pi
        
        output['pt'].fill(dataset=dataset,pt=Jet0.p4.pt.flatten())
        output['eta'].fill(dataset=dataset,eta=Jet0.p4.eta.flatten())
        output['dr'].fill(dataset=dataset,dr=dr.flatten())
        if ("ZZbbbb" in dataset):
            output['h_drjb'].fill(dataset=dataset,dr=dRMatch_jb.flatten())
            #output['h_drjbb'].fill(dataset=dataset,dr=dRMatched_jb.flatten())
            #output['h_drbb'].fill(dataset=dataset,dr=dRMatch_bb.flatten())
        output['h_dxy'].fill(dataset=dataset,dxy=np.sqrt(np.power(truncated_constituents.vxx, 2) + np.power(truncated_constituents.vxx, 2)).flatten())
        output['h_dz'].fill(dataset=dataset,dz=truncated_constituents.vzz.flatten())
        output['pdgid'].fill(dataset=dataset,pdgid=jetconstituents.pdgid.flatten())
        output['h_hadmult'].fill(dataset=dataset,mult=isCh.sum(),hadron="Charged Hadrons")
        output['h_hadmult'].fill(dataset=dataset,mult=isNh.sum(),hadron="Neutral Hadrons")
        output['h_phoMult'].fill(dataset=dataset,mult=isPho.sum())        
        output['h_eleMult'].fill(dataset=dataset, mult=isEle.sum())          
        output['h_muMult'].fill(dataset=dataset,mult=isMuon.sum())
        output['image'].fill(dataset=dataset,
                            event=eventReshaped.flatten(), 
                            image_deta=deta.flatten(),
                            image_dphi=dphi.flatten(),
                            weight=normed_pts.flatten())
        
        output['h_jetPull'].fill(dataset=dataset,pull=pull_mag.flatten())
        output['h_jetPullAngle'].fill(dataset=dataset,pullAngle=pull_angle.flatten())
        
        output['flippedImages'] += processor.column_accumulator(H_images_new[validImages])
        output['jetPt'] += processor.column_accumulator(Jet0.p4.pt.flatten())
        output['jetEta'] += processor.column_accumulator(Jet0.p4.eta.flatten())
        output['jetPhi'] += processor.column_accumulator(Jet0.p4.phi.flatten())
        output['jetMass'] += processor.column_accumulator(Jet0.p4.mass.flatten())
        output['jetMassSD'] += processor.column_accumulator(Jet0.msd.flatten())


# #         output['jetconstEvnum'] += processor.column_accumulator(np.repeat(eventReshaped[:,:1].flatten(), 20, axis=0).reshape(len(eventReshaped[:,:1]),20))
# #         output['jetconstPt'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.pt/Jet0.p4.pt).fillna(0.0).values)
        output['jetconstPt'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.pt).fillna(0.0).values)
        output['jetconstPt_log'] += processor.column_accumulator(pd.DataFrame(np.log(truncated_constituents.p4.pt)).fillna(0.0).values)
        output['jetconstPt_Jetlog'] += processor.column_accumulator(pd.DataFrame(np.log(truncated_constituents.p4.pt/Jet0.p4.pt)).fillna(0.0).values)
        output['jetconstEta'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.eta).fillna(-99).values)
        output['jetconstPhi'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.phi).fillna(-99).values)
        output['jetconstE'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.energy).fillna(0.0).values)
        output['jetconstE_log'] += processor.column_accumulator(pd.DataFrame(np.log(truncated_constituents.p4.energy)).fillna(0.0).values)
        output['delta_eta'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.eta - Jet0.p4.eta).fillna(-99).values)
        output['delta_phi'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.delta_phi(Jet0.p4)).fillna(-99).values)
        output['deltaR_jet'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.delta_r(Jet0.p4)).fillna(-99).values)
        output['deltaR_subjet0'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.delta_r(SubJet0)).fillna(-99).values)
        output['deltaR_subjet1'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.p4.delta_r(SubJet1)).fillna(-99).values)
        output['charge'] += processor.column_accumulator(pd.DataFrame(charge).fillna(-99).values)
        output['isEle'] += processor.column_accumulator(pd.DataFrame(t_isEle).fillna(False).values)
        output['isMuon'] += processor.column_accumulator(pd.DataFrame(t_isMuon).fillna(False).values)
        output['isPho'] += processor.column_accumulator(pd.DataFrame(t_isPho).fillna(False).values)
        output['isCh'] += processor.column_accumulator(pd.DataFrame(t_isCh).fillna(False).values)
        output['isNh'] += processor.column_accumulator(pd.DataFrame(t_isNh).fillna(False).values)
        output['dz'] += processor.column_accumulator(pd.DataFrame(truncated_constituents.vzz).fillna(-99).values)
        output['dxy'] += processor.column_accumulator(pd.DataFrame(np.sqrt(np.power(truncated_constituents.vxx, 2) + np.power(truncated_constituents.vxx, 2))).fillna(-99).values)
        output['chargedHadronMult'] += processor.column_accumulator(isCh.sum().flatten())
        output['neutralHadronMult'] += processor.column_accumulator(isNh.sum().flatten())
        output['photonMult'] += processor.column_accumulator(isPho.sum().flatten())
        output['electronMult'] += processor.column_accumulator(isEle.sum().flatten())
        output['muonMult'] += processor.column_accumulator(isMuon.sum().flatten())
        output['pull_angle'] += processor.column_accumulator(pull_angle.flatten())
        output['deltaR_subjets'] += processor.column_accumulator(SubJet0.delta_r(SubJet1).flatten())
        output['z'] += processor.column_accumulator(SubJet0.p.flatten() / (SubJet0.p.flatten() + SubJet1.p.flatten()))
        output['label'] += processor.column_accumulator(labels)
        
        
        for itau_beta, tau_beta in enumerate(self.tau_betas):
            # Adding n-subjettiness information
            tau1 = Jet0.tau1[:,itau_beta].flatten()
            tau2 = Jet0.tau2[:,itau_beta].flatten()
            tau3 = Jet0.tau3[:,itau_beta].flatten()
            # tau4 = Jets.tau4[:,0].flatten()
            tau1_sd = Jet0.tau1_sd[:,itau_beta].flatten()
            tau2_sd = Jet0.tau2_sd[:,itau_beta].flatten()
            tau3_sd = Jet0.tau3_sd[:,itau_beta].flatten()
            # tau4 = Jets.tau4[:,0].flatten()
            
            output['jetTau1_b' + str(tau_beta)] += processor.column_accumulator(tau1)
            output['jetTau2_b' + str(tau_beta)] += processor.column_accumulator(tau2)
            output['jetTau3_b' + str(tau_beta)] += processor.column_accumulator(tau3)
            output['jetTau1_sd_b' + str(tau_beta)] += processor.column_accumulator(tau1_sd)
            output['jetTau2_sd_b' + str(tau_beta)] += processor.column_accumulator(tau2_sd)
            output['jetTau3_sd_b' + str(tau_beta)] += processor.column_accumulator(tau3_sd)


            
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [6]:
runstring = 'addmoretaus'
outstring = '_ZZ'

In [7]:
fileset={    
#    'ZZ':['/mnt/data/ml/zz_flatter_%s.root' % (runstring)],
#     'ZZbbbb':['/mnt/data/ml/zz_bb_flatter_addmoretaus.root'],
#     'QCD':['/mnt/data/ml/qcd_flatter50to7000_%s.root' % (runstring)]
}

fileset = {'QCD':[
#                   '/mnt/data/ml/qcd_flatter50to7000_ext4.root',
#                   '/mnt/data/ml/qcd_flatter50to7000_ext5.root',
#                   '/mnt/data/ml/qcd_flatter50to7000_ext6.root',
#                   '/mnt/data/ml/qcd_flatter50to7000_ext7.root',
                  '/mnt/data/ml/qcd_flatter50to7000_ext8.root',
#                   '/mnt/data/ml/qcd_flatter50to7000_ext9.root',
#                   '/mnt/data/ml/qcd_flatter50to7000_ext10.root',
                 ]
          }



In [8]:



tstart = time.time() 
output = processor.run_uproot_job(fileset,
                                  treename='T',
                                  processor_instance=JetImageProcessor(),
                                  executor=processor.futures_executor,
                                  executor_args={'workers':4, 'flatten': True},
                                  chunksize=5000#, maxchunks=5
                                 )    
elapsed = time.time() - tstart

# print(output)

Process Process-1:
Traceback (most recent call last):
  File "/usr/lib64/python3.6/concurrent/futures/process.py", line 175, in _process_worker
    r = call_item.fn(*call_item.args, **call_item.kwargs)
  File "/home/jovyan/coffea/coffea/processor/executor.py", line 126, in _work_function
    out = processor_instance.process(df)
  File "<ipython-input-5-2966fc294d54>", line 262, in process
    bins=(eventbins ,ybins, xbins), weights = normed_pts.flatten())[0]
  File "<__array_function__ internals>", line 6, in histogramdd
  File "/usr/local/jupyter/lib/python3.6/site-packages/numpy/lib/histograms.py", line 1079, in histogramdd
    hist = np.bincount(xy, weights, minlength=nbin.prod())
  File "<__array_function__ internals>", line 6, in bincount
MemoryError: Unable to allocate array with shape (1466104212,) and data type float64

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib64/python3.6/multiprocessing/process.py"

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
dataStream(output, runstring + outstring)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [ ]:
plt.rcParams['figure.dpi'] = 100

In [ ]:
fig, ax, grid = hist.plot2d(output['image'].integrate('dataset', 'ZZ').integrate("event",1),
                            xaxis='image_deta', patch_opts={'norm':LogNorm(vmin=1, vmax=1e3),'cmap':'Greens'})
print(fig)
fig.savefig("ZZ_eventImage.png")

In [ ]:
fig, ax, grid = hist.plot2d(output['image'].integrate('dataset', 'ZZbbbb').integrate("event",1),
                            xaxis='image_deta', patch_opts={'norm':LogNorm(vmin=1, vmax=1e3),'cmap':'Greens'})
print(fig)
fig.savefig("ZZbbbb_eventImage.png")

In [ ]:
fig, ax, grid = hist.plot2d(output['image'].integrate('dataset', 'ZZ').integrate("event"),
                            xaxis='image_deta', patch_opts={'norm':LogNorm(vmin=1, vmax=1e3),'cmap':'Greens'})
print(fig)
fig.savefig("ZZ_image.png")

In [ ]:
fig, ax, grid = hist.plot2d(output['image'].integrate('dataset', 'QCD').integrate("event",1),
                            xaxis='image_deta', patch_opts={'norm':LogNorm(vmin=1, vmax=1e3),'cmap':'Greens'})
print(fig)
fig.savefig("QCD_eventImage.png")

In [ ]:
fig, ax, grid = hist.plot2d(output['image'].integrate('dataset', 'QCD').integrate("event"),
                            xaxis='image_deta', patch_opts={'norm':LogNorm(vmin=1, vmax=1e3),'cmap':'Greens'})
print(fig)
fig.savefig("QCD_image.png")

In [ ]:
fig, ax = hist.plotgrid(output['dr'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()
plt.yscale('log')
fig.savefig("dr_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_drjb'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()
plt.yscale('log')
fig.savefig("drjb_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_dxy'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()
plt.yscale('log')
plt.xscale('log')
fig.savefig("dxy_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_dz'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()
plt.yscale('log')
#plt.xscale('log')
fig.savefig("dz_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['pt'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
fig.show()
fig.savefig("pt_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_hadmult'].integrate("hadron","Charged Hadrons"), overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
    iax.set_xlabel("Charged Hadron Multicplicity")
    
fig.show()
fig.savefig("chMult_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_hadmult'].integrate("hadron","Neutral Hadrons"), overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
    iax.set_xlabel("Neutral Hadron Multicplicity")
    
fig.show()
fig.savefig("neutMult_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_eleMult'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
    iax.set_xlim([0,10])
#     iax.set_title('Electron Multiplicity')
    iax.set_xlabel('Electron Multiplicity')

fig.show()
fig.savefig("eleMult_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_muMult'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
    iax.set_xlim([0,10])
#     iax.set_title('Muon Multiplicity')
    iax.set_xlabel('Muon Multiplicity')

fig.show()
fig.savefig("muMult_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_phoMult'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
#     iax.set_title('Photon Multiplicity')
    iax.set_xlabel('Photon Multiplicity')



fig.show()
fig.savefig("phoMult_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_jetPullAngle'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
    iax.set_title('Jet Pull Angle')
    iax.grid(True)


fig.show()
fig.savefig("pullAngle_plot.png")

In [ ]:
fig, ax = hist.plotgrid(output['h_jetPull'], overlay="dataset", stack=False, density=True
                                 )
for iax in ax.flatten():
    iax.autoscale(axis='y')
#     iax.set_title('Jet Pull')
    iax.set_xlabel('Jet Pull')
    iax.set_xlim([0,0.01])

fig.show()
fig.savefig("jetPull_plot.png")